In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from matplotlib import pyplot as plt
import sys

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set

from bo_methods_lib.CS2_bo_plotters import plot_obj_abs_min, value_plotter, plot_obj, plot_Theta, plot_Theta_min,path_name, plot_org_train, save_fig, plot_EI_abs_max

from bo_methods_lib.CS2_bo_functions_multi_dim import set_ep

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html

def csv_to_array(csv_path):
    """
    Turn a csv file into a numpy array so that it can be used with bo_plotters
    
    Parameters:
    -----------
        csv_path: str, path of csv you want to turn into an array
    Returns:
    --------
        csv_array: np.ndarray, array on values in the CSV file
    """
    
    csv_pd = pd.read_csv(csv_path, index_col = False)
    csv_pd.drop(columns=csv_pd.columns[0], 
        axis=1, 
        inplace=True)
    csv_array = csv_pd.to_numpy()
    
    return csv_array

In [3]:
#WORKS
#Set Date and Time
DateTime = "2022/10/21/13-18" #Date and Time - Dispayed in job file
# DateTime = None ##For Testing

#Set Parameters
exp_d = 1
n = 5
t_org = 20
tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
tot_runs = 15 #Total Number of runs - Dispayed in job file
sep_fact = np.linspace(0.1,1,1)
set_lengthscale = None
is_figure = False
save_figure = False
save_CSV = False


param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
indecies = [list(param_dict.keys())[list(param_dict.values()).index('\\theta_1')],
            list(param_dict.keys())[list(param_dict.values()).index('\\theta_2')]]

Theta_True = np.array([1,-1])

bounds = np.array([[-2, -2],
                   [ 2,  2]])

param_names_list = [param_dict[0], param_dict[1]] #Fill in depending on problem
mesh_combo = None

d = len(Theta_True)

obj_list = np.array(["obj","LN_obj"])
emulator_list = np.array([False, True])
sparse_grid_list = np.array([False,True])

df_list = []
print("Runs:", tot_runs)
print("BO Iterations:",tot_iter)
print("%%%%%%%%%%%%%%%%%%%%%%%%%%")
count = 0
for emulator in emulator_list: 
    sys.stdout.flush()
    print("-------------------")
    print("Emulator?:", emulator)
    if emulator == True:
        t = n*t_org
        sparse_grid_use = sparse_grid_list
    else:
        t = t_org
        sparse_grid_use = np.array([sparse_grid_list[0]]) #Sparse Grid will always be False for 2-Input
        
    for sparse_grid in sparse_grid_use:
#         #Can set ep to 1 for sparse grid if wanted
        if sparse_grid == True:
            obj_use =  np.array(["obj"])
        else:
            obj_use =  obj_list
#             ep_use = torch.tensor([1]) 
#         else:
#             ep_use = explore_bias
        print("______________________________")
        print("Sparse Grid?:", sparse_grid)  
        
        for obj in obj_use:      
            print("Objective Function:", obj)
            print("-  -  -  -  -  -  -  -  -  -  -")

            ep = set_ep(emulator, obj, sparse_grid)
            
            array_info = []
            sep_fact_list = np.linspace(0.1,1,10)
            for k in range(len(sep_fact_list)):     
                csv_end = ".csv" #For obj
                fxn_obj = "plot_obj"
                fxn_obj_min = "plot_obj_abs_min"
                path_obj = "../" + path_name(emulator, ep, sparse_grid, fxn_obj, set_lengthscale, t, obj, mesh_combo = None, 
                                             bo_iter= None, title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, 
                                             DateTime = DateTime, sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
                path_obj_min = "../" + path_name(emulator, ep, sparse_grid, fxn_obj_min, set_lengthscale, t, obj, mesh_combo = None, 
                                             bo_iter= None, title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, 
                                             DateTime = DateTime, sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
                obj_array = csv_to_array(path_obj)
                obj_array_min = csv_to_array(path_obj_min)

                for i in range(tot_runs): 
                    obj_df_run = pd.DataFrame(data = obj_array[i])
                    obj_df_i = obj_df_run.loc[(abs(obj_df_run) > 1e-6).any(axis=1),0]
                    max_iter = len(obj_df_i)
                    obj_no_zero_arr = list(np.array(obj_df_i).flatten())

                    obj_df_run_min = pd.DataFrame(data = obj_array_min[i])
                    obj_df_i_min = obj_df_run_min.loc[(abs(obj_df_run_min) > 1e-6).any(axis=1),0]
                    max_iter_min = len(obj_df_i_min)
                    obj_no_zero_arr = list(np.array(obj_df_i_min).flatten())
                    for j in range(len(obj_df_i)):
                        array_info.append([np.round(sep_fact_list[k],1), i+1, j+1,np.exp(obj_df_i[j]), len(obj_df_i), 
                                           np.exp(obj_df_i_min[j])])

            df = pd.DataFrame(data = array_info, columns = ["Separation Factor", "Run", "Iteration", "SSE", "Max Eval", "Minimum SSE"])

            for k in range(len(df)): 
                fxn_theta = "plot_Theta"
                for i in range(len(param_dict.keys())):
                    csv_end = "/"+param_dict[i]+ ".csv" #For Thetas #Need to ensure this saves correctly

                    path_i = "../" + path_name(emulator, ep, sparse_grid, fxn_theta, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
                              title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                                     sep_fact = df["Separation Factor"][k], is_figure = is_figure, csv_end = csv_end)
                    path_i = path_i.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv/Theta_Conv_"+str(i+1))
                    theta_i = csv_to_array(path_i)
            #         print(df["Run"][k], df["Iteration"][k])
                    theta_ij = theta_i[df["Iteration"][k]-1, df["Run"][k]-1]
                    df.loc[df.index[k], "Theta " + str(i+1)] = theta_ij
                    
                    path_i_min = "../" + path_name(emulator, ep, sparse_grid, fxn_theta, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
                  title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                         sep_fact = df["Separation Factor"][k], is_figure = is_figure, csv_end = csv_end)
                    path_i_min = path_i_min.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv_min/Theta_Conv_min"+str(i+1))
                    theta_i_min = csv_to_array(path_i_min)
            #         print(df["Run"][k], df["Iteration"][k])
                    theta_ij_min = theta_i_min[df["Iteration"][k]-1, df["Run"][k]-1]
                    df.loc[df.index[k], "Min Theta " + str(i+1)] = theta_ij_min
            #         Theta_info = theta_i[]

            # print(df[20:40])
           
            df_list.append(df)

Runs: 15
BO Iterations: 100
%%%%%%%%%%%%%%%%%%%%%%%%%%
-------------------
Emulator?: False
______________________________
Sparse Grid?: False
Objective Function: obj
-  -  -  -  -  -  -  -  -  -  -
Objective Function: LN_obj
-  -  -  -  -  -  -  -  -  -  -
-------------------
Emulator?: True
______________________________
Sparse Grid?: False
Objective Function: obj
-  -  -  -  -  -  -  -  -  -  -
Objective Function: LN_obj
-  -  -  -  -  -  -  -  -  -  -
______________________________
Sparse Grid?: True
Objective Function: obj
-  -  -  -  -  -  -  -  -  -  -


In [4]:
method_names = ["1A", "1B", "2A", "2B", "2C"] 
method_times = [0.0547, 0.100, 0.392, 4.36, 0.615] 

for i in range(len(df_list)):
    df_list[i].loc[:,"Time/Iter (Minutes)"] = method_times[i]
    df_list[i].loc[:,"Total Run Time"] = df_list[i].loc[:,"Max Eval"]*df_list[i].loc[:,"Time/Iter (Minutes)"]
    df_list[i].loc[:,"Method"] = method_names[i]
    cols = df_list[i].columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df_list[i] = df_list[i][cols]
    print(df_list[i].head())

  Method  Separation Factor  Run  Iteration        SSE  Max Eval  Minimum SSE  \
0     1A                0.1    1          1  43.893042       100    43.893042   
1     1A                0.1    1          2   7.522665       100     7.522665   
2     1A                0.1    1          3   7.481745       100     7.481745   
3     1A                0.1    1          4   4.540652       100     4.540652   
4     1A                0.1    1          5   2.320721       100     2.320721   

    Theta 1  Min Theta 1   Theta 2  Min Theta 2  Time/Iter (Minutes)  \
0  2.000000     2.000000 -2.000000    -2.000000               0.0547   
1  1.412874     1.412874 -0.591451    -0.591451               0.0547   
2  1.561159     1.561159 -0.649155    -0.649155               0.0547   
3  1.402607     1.402607 -0.712744    -0.712744               0.0547   
4  1.206524     1.206524 -1.237808    -1.237808               0.0547   

   Total Run Time  
0            5.47  
1            5.47  
2            5.47  


In [5]:
df_all = pd.concat(df_list)
df_all_sorted = df_all.sort_values(by=['Separation Factor', 'Method'], ascending=True)

In [6]:
df_all.to_csv("CS1_results_by_iter.csv")